**Question 4**

[Google Colab](https://colab.research.google.com/drive/18UVsYGcFUgsWPmJYAd8jb6iglFKcYTqx?usp=sharing)

In [1]:
# Clone necessary repositories
!git clone https://github.com/zhouhaoyi/Informer2020.git
!git clone https://github.com/zhouhaoyi/ETDataset.git

Cloning into 'Informer2020'...
remote: Enumerating objects: 576, done.
remote: Counting objects: 100% (222/222), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 576 (delta 199), reused 188 (delta 188), pack-reused 354
Receiving objects: 100% (576/576), 6.48 MiB | 10.86 MiB/s, done.
Resolving deltas: 100% (336/336), done.
Cloning into 'ETDataset'...
remote: Enumerating objects: 187, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 187 (delta 25), reused 20 (delta 20), pack-reused 159
Receiving objects: 100% (187/187), 3.86 MiB | 7.19 MiB/s, done.
Resolving deltas: 100% (62/62), done.


In [2]:
import sys
if not 'Informer2020' in sys.path:
    sys.path += ['Informer2020']

In [3]:
# Import required modules and libraries
from utils.tools import dotdict
from exp.exp_informer import Exp_Informer
import torch
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [4]:
# Define configuration settings
args = dotdict()
args.model = 'informer'
args.data = 'ETTh1'
args.root_path = './ETDataset/ETT-small/'
args.data_path = 'ETTh1.csv'
args.features = 'M'
args.target = 'OT'
args.freq = 'h'
args.checkpoints = './informer_checkpoints'

args.seq_len = 96
args.label_len = 48
args.pred_len = 24

args.enc_in = 7
args.dec_in = 7
args.c_out = 7
args.factor = 5
args.d_model = 512
args.n_heads = 8
args.e_layers = 2
args.d_layers = 1
args.d_ff = 2048
args.dropout = 0.05
args.embed = 'timeF'
args.activation = 'gelu'
args.distil = True
args.output_attention = False
args.mix = True
args.padding = 0
args.freq = 'h'

args.batch_size = 32
args.learning_rate = 0.0001
args.loss = 'mse'
args.lradj = 'type1'
args.use_amp = False

args.num_workers = 0
args.itr = 1
args.train_epochs = 6
args.patience = 3
args.des = 'exp'

args.use_gpu = True if torch.cuda.is_available() else False
args.gpu = 0

args.use_multi_gpu = False
args.devices = '0,1,2,3'


In [5]:
# Set the attention type to 'prob' for Probsparse attention
args.attn = 'prob'

# Define a function to train and test the model
def train_and_test(model_type):
    setting = f'{model_type}'

    exp = Exp_Informer(args)

    print(f'---------------------- Start training: {setting} ---------------------->')
    exp.train(setting)

    print(f'---------------------- Testing: {setting} ----------------------')
    exp.test(setting)

# Train and test the Probsparse attention model
train_and_test('Probsparse')


Use GPU: cuda:0
---------------------- Start training: Probsparse ---------------------->
train 8521
val 2857
test 2857
	iters: 100, epoch: 1 | loss: 0.4764659
	speed: 0.1481s/iter; left time: 221.7066s
	iters: 200, epoch: 1 | loss: 0.3557161
	speed: 0.0683s/iter; left time: 95.3603s
Epoch: 1 cost time: 25.803321599960327
Epoch: 1, Steps: 266 | Train Loss: 0.4169796 Vali Loss: 0.6821633 Test Loss: 0.5922198
Validation loss decreased (inf --> 0.682163).  Saving model ...
Updating learning rate to 0.0001
	iters: 100, epoch: 2 | loss: 0.2703295
	speed: 0.1625s/iter; left time: 199.9824s
	iters: 200, epoch: 2 | loss: 0.1978941
	speed: 0.0701s/iter; left time: 79.2286s
Epoch: 2 cost time: 18.69873809814453
Epoch: 2, Steps: 266 | Train Loss: 0.2502195 Vali Loss: 0.6543479 Test Loss: 0.5604963
Validation loss decreased (0.682163 --> 0.654348).  Saving model ...
Updating learning rate to 5e-05
	iters: 100, epoch: 3 | loss: 0.2019773
	speed: 0.1667s/iter; left time: 160.8967s
	iters: 200, epoch

In [6]:
# Set the attention type to 'full' for Canonical attention
args.attn = 'full'

# Train and test the Canonical attention model
train_and_test('Canonical')

Use GPU: cuda:0
---------------------- Start training: Canonical ---------------------->
train 8521
val 2857
test 2857
	iters: 100, epoch: 1 | loss: 0.5083330
	speed: 0.0666s/iter; left time: 99.7370s
	iters: 200, epoch: 1 | loss: 0.2721415
	speed: 0.0670s/iter; left time: 93.5705s
Epoch: 1 cost time: 17.792943477630615
Epoch: 1, Steps: 266 | Train Loss: 0.3756041 Vali Loss: 0.6687545 Test Loss: 0.6176901
Validation loss decreased (inf --> 0.668754).  Saving model ...
Updating learning rate to 0.0001
	iters: 100, epoch: 2 | loss: 0.2560747
	speed: 0.1492s/iter; left time: 183.6377s
	iters: 200, epoch: 2 | loss: 0.2679474
	speed: 0.0671s/iter; left time: 75.9449s
Epoch: 2 cost time: 17.842905521392822
Epoch: 2, Steps: 266 | Train Loss: 0.2380875 Vali Loss: 0.6515129 Test Loss: 0.5402398
Validation loss decreased (0.668754 --> 0.651513).  Saving model ...
Updating learning rate to 5e-05
	iters: 100, epoch: 3 | loss: 0.1508450
	speed: 0.1507s/iter; left time: 145.3874s
	iters: 200, epoch:

In [7]:
probsparse_ground_truth = np.load('./results/Probsparse/true.npy')
probsparse_predictions = np.load('./results/Probsparse/pred.npy')

# Ensure both arrays have the same shape by slicing to match the dimensions
probsparse_ground_truth = probsparse_ground_truth[0, -args.pred_len:]
probsparse_predictions = probsparse_predictions[0, -args.pred_len:]

# Calculate MSE and MAE for the Probsparse attention model
probsparse_mse = mean_squared_error(probsparse_ground_truth, probsparse_predictions)
probsparse_mae = mean_absolute_error(probsparse_ground_truth, probsparse_predictions)

print(f'MSE for Probsparse Attention Model: {probsparse_mse}')
print(f'MAE for Probsparse Attention Model: {probsparse_mae}')

MSE for Probsparse Attention Model: 0.507151186466217
MAE for Probsparse Attention Model: 0.48420387506484985


In [8]:
canonical_ground_truth = np.load('./results/Canonical/true.npy')
canonical_predictions = np.load('./results/Canonical/pred.npy')

# Ensure both arrays have the same shape by slicing to match the dimensions
canonical_ground_truth = canonical_ground_truth[0, -args.pred_len:]
canonical_predictions = canonical_predictions[0, -args.pred_len:]

# Calculate MSE and MAE for the Canonical attention model
canonical_mse = mean_squared_error(canonical_ground_truth, canonical_predictions)
canonical_mae = mean_absolute_error(canonical_ground_truth, canonical_predictions)

print(f'MSE for Canonical Attention Model: {canonical_mse}')
print(f'MAE for Canonical Attention Model: {canonical_mae}')

MSE for Canonical Attention Model: 0.5316560864448547
MAE for Canonical Attention Model: 0.5037769079208374
